## 1. import dataset

In [3]:
import zipfile
import pandas as pd
import os
import re

# Directory where the ZIP files are located (assuming it's relative)
base_directory = '../data/raw/itineraries_csv'  

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Define a regular expression pattern to match filenames ending with two lowercase letters
file_pattern = re.compile(r'.*[a-z][a-z]\.zip')

# List all airport folders in the base directory
airport_names = [name for name in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, name))]

dfs = []  # Create a list to store DataFrames

for airport_name in airport_names:
    # Directory path for the current airport
    zip_directory = os.path.join(base_directory, airport_name)
    
    # List all files in the airport's folder
    file_list = os.listdir(zip_directory)

    for filename in file_list:
        if file_pattern.match(filename):
            zip_file_path = os.path.join(zip_directory, filename)
            csv_file_path_inside_zip = filename.replace('.zip', '.csv')
            
            # Create a ZipFile object and read the CSV file
            with zipfile.ZipFile(zip_file_path, 'r') as zf:
                df = pd.read_csv(zf.open(csv_file_path_inside_zip))

            # Append the DataFrame for this filename to the list of DataFrames
            dfs.append(df)

# Use pandas.concat to concatenate the list of DataFrames into a single DataFrame
all_airport = pd.concat(dfs, ignore_index=True)

KeyboardInterrupt: 

In [ ]:

# legId                                      0
# searchDate                                0
# flightDate                                0
# startingAirport                           0
# destinationAirport                        0
# travelDuration                            0
# isBasicEconomy                            0
# isRefundable                              0
# totalFare                                 0 (y)
# totalTravelDistance                  959619
# segmentsDepartureTimeEpochSeconds         0
# segmentsArrivalTimeEpochSeconds           0
# segmentsArrivalAirportCode                0
# segmentsDepartureAirportCode              0
# segmentsAirlineCode                       0
# segmentsEquipmentDescription         262676
# segmentsDurationInSeconds                 0 -> sum
# segmentsDistance                          0 -> sum
# segmentsCabinCode                         0
# -------
# travelLayover (travelDuration - segmentsDurationInSeconds)
# datediff (flightDate - searchDate)
# transitAirportCode (list) -> check arrival departure 
# numberOfTransit -> count (transitAirportCode)

## 2. Data preprocessing

### Create `travelLayover` column

**Convert `travelDuration` into second**

In [ ]:
# Function to convert a duration string to seconds
def convert_duration_to_seconds(duration):
    match = re.match(r'PT(\d+)H(\d+)M', duration)
    
    if match:
        hours = int(match.group(1))
        minutes = int(match.group(2))
        total_seconds = hours * 3600 + minutes * 60
        return total_seconds
    else:
        return None

# Apply the function to the 'travelDuration' column and create a new column 'travelDurationInSeconds'
all_airport['travelDurationInSeconds'] = all_airport['travelDuration'].apply(convert_duration_to_seconds)


In [ ]:
all_airport['travelDurationInSeconds'].head()

0    28320.0
1    22500.0
2    32760.0
3    22620.0
4    51120.0
Name: travelDurationInSeconds, dtype: float64

**Convert `durationinsecond`**`

In [ ]:
# Function to split and sum the values
def split_and_sum(segment_duration):
    segments = segment_duration.split('||')
    return sum(map(int, segments))

# Apply the function to the 'segmentsDurationInSeconds' column and create a new column 'totalDurationInSeconds'
all_airport['totalDurationInSeconds'] = all_airport['segmentsDurationInSeconds'].apply(split_and_sum)


In [ ]:
all_airport['totalDurationInSeconds'].head()

0    19800
1    20520
2    20520
3    19560
4    25080
Name: totalDurationInSeconds, dtype: int64

**Calculate `travelLayover`**

In [ ]:
all_airport['travelLayover'] = all_airport['travelDurationInSeconds'] - all_airport['totalDurationInSeconds']
all_airport['travelLayover'].head()

0     8520.0
1     1980.0
2    12240.0
3     3060.0
4    26040.0
Name: travelLayover, dtype: float64

In [ ]:
### Find Sum of duration
all_airport['SumsegmentsDurationInSeconds'] = all_airport['segmentsDurationInSeconds'].str.split(r'\|\|').apply(lambda x: [pd.to_numeric(value, errors='coerce') for value in x])
all_airport['SumsegmentsDurationInSeconds'] = all_airport['SumsegmentsDurationInSeconds'].apply(sum)

NameError: name 'all_airport' is not defined

In [ ]:
#sepearte col by ||
all_airport['segmentsArrivalAirportCode'] = all_airport['segmentsArrivalAirportCode'].str.split(r'\|\|')
all_airport['segmentsDepartureAirportCode'] = all_airport['segmentsDepartureAirportCode'].str.split(r'\|\|')
#find departure and arrival
all_airport['Departure'] = all_airport['segmentsDepartureAirportCode'].apply(lambda x: x[0])
all_airport['Arrival'] = all_airport['segmentsArrivalAirportCode'].apply(lambda x: x[-1])

In [ ]:
#create new col to collect only transit airport
import pandas as pd

# Assuming all_airport is your DataFrame

def process_code_list(code_list):
    if len(code_list) != 1:
        code_list = code_list[1:] 
    return code_list

all_airport['transitAirportCode'] = all_airport['segmentsDepartureAirportCode'].apply(process_code_list)

In [ ]:
#create a new col to collect all airport
all_airport['AllAirport'] = all_airport['transitAirportCode']
all_airport['AllAirport'] = all_airport.apply(lambda row: [row['Departure']] + row['AllAirport'], axis=1)
all_airport['AllAirport'] = all_airport.apply(lambda row: row['AllAirport'] + [row['Arrival']], axis=1)